In [1]:
import numpy as np
import pandas as pd
import torch
import sys
code_path = '/home/user-home/yujie/0_PBCNetv2/0_PBCNET'
sys.path.append(f'{code_path}/model_code')
from torch.utils.data import DataLoader
from tqdm import tqdm
from utilis.utilis import pkl_load


In [2]:
def minmax(tensor_x):
    min_val = torch.min(tensor_x)
    max_val = torch.max(tensor_x)
    tensor_x_normalized = (tensor_x - min_val) / (max_val - min_val)

    return tensor_x_normalized

In [3]:
model = torch.load(f"{code_path}//PBCNet2.pth", map_location=torch.device('cpu'), weights_only=False)

In [4]:
# sys = f"{code_path}/data/Mutation/P00469/WT_1THY_ligand.pkl"
sys = f'{code_path}/data/Mutation/P00469/MT_1THY_N229C_ligand.pkl'
g = pkl_load(sys)
index = np.where(g.nodes['atom'].data['x'] == 7)[0][1]
index

2

In [5]:
g.nodes['atom'].data['pos'][index]

tensor([ 13.8244,  46.3555, -50.7142])

In [6]:
def mean_sca(att, type):
    input = torch.zeros(torch.max(type)+1)
    v = input.scatter_reduce_(0, type, att, reduce="mean", include_self=False)
    return v

In [7]:

g.edges['int'].data['bond_scalar'] = torch.clip(g.edges['int'].data['bond_scalar'].long(), min=0, max=4)
g.edges['ind'].data['bond_scalar'] = torch.clip(g.edges['ind'].data['bond_scalar'].long(), min=0, max=4)
emb,att = model._readout(g, bb=False)
att =att[0]
bond_type = 'int'
print(att.shape)
att1 = att.reshape(att.shape[0], -1)

att1_ = torch.mean(att1,dim=-1)
mask = g.nodes['atom'].data['type'].detach().numpy()

def src_norm_dst(src_field, dst_field, out_field):
    def func(edges):
        return {out_field: torch.sqrt(torch.sum((edges.src[src_field] - edges.dst[dst_field]) ** 2, dim=1))}
    return func
g.apply_edges(src_norm_dst('pos', 'pos', 'dist'), etype=bond_type)
d = g.edges[bond_type].data['dist']
type = g.edges[bond_type].data['bond_scalar'].long()

a = mean_sca(att1_, type)
print(a)



torch.Size([2590, 96, 3])
tensor([0.0096, 0.0096, 0.0000, 0.0121, 0.0160],
       grad_fn=<ScatterReduceBackward0>)


In [8]:
g = pkl_load(sys)
a = np.intersect1d(np.where(g.edges['int'].data['bond_scalar'].long() == 6)[0], np.where(g.edges(etype = 'int')[1].detach().numpy() == index)[0])
b = np.intersect1d(np.where(g.edges['int'].data['bond_scalar'].long() == 6)[0], np.where(g.edges(etype = 'int')[0].detach().numpy() == index)[0])
minmax(att1_[b])

tensor([0.0000, 0.4341, 0.0591, 0.8780, 0.6908, 1.0000, 0.6187],
       grad_fn=<DivBackward0>)

In [9]:
g.nodes['atom'].data['pos'][g.edges(etype = 'int')[0].detach().numpy()[a]]

tensor([[ 11.0640,  48.8600, -53.3720],
        [ 12.5460,  49.4680, -54.2140],
        [  9.4040,  47.3100, -48.7640],
        [ 10.7100,  44.4150, -52.0210],
        [ 11.5710,  45.4160, -54.7070],
        [ 15.4990,  42.6460, -48.4710],
        [ 11.1600,  44.2070, -48.3670]])

In [10]:
g.nodes['atom'].data['x'][g.edges(etype = 'int')[0].detach().numpy()[a]]

tensor([ 6, 16,  6,  7,  8,  6, 16])

In [2]:
import pandas as pd
a = pd.read_csv("/home/user-home/yujie/0_PBCNetv2/data_BindingDB/pdbs/training_clip_862W.csv")
a.dir_2.values

array(['/home/user-home/yujie/0_PBCNetv2/data_BindingDB/pdbs/sys_graph/bindingDB_107668/lig30.pkl',
       '/home/user-home/yujie/0_PBCNetv2/data_BindingDB/pdbs/sys_graph/bindingDB_87605/lig1.pkl',
       '/home/user-home/yujie/0_PBCNetv2/data_BindingDB/pdbs/sys_graph/bindingDB_61468/lig9.pkl',
       ...,
       '/home/user-home/yujie/0_PBCNetv2/data_BindingDB/pdbs/sys_graph/bindingDB_79658/lig18.pkl',
       '/home/user-home/yujie/0_PBCNetv2/data_BindingDB/pdbs/sys_graph/bindingDB_107056/lig125.pkl',
       '/home/user-home/yujie/0_PBCNetv2/data_BindingDB/pdbs/sys_graph/bindingDB_62106/lig185.pkl'],
      dtype=object)